<a href="https://colab.research.google.com/github/teamgaon/KIST/blob/main/220504_img_masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import albumentations
import albumentations.pytorch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
import os
import json
import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
import albumentations as A
from sklearn.model_selection import KFold, StratifiedKFold
from datetime import datetime, timedelta
import random
import torchvision.transforms as transforms
import gc
from pathlib import Path

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
images = glob('/content/drive/MyDrive/KIST/open/train/*/image/*')
images.sort()
metas  = glob('/content/drive/MyDrive/KIST/open/train/*/meta/*')
metas.sort()
labels = glob('/content/drive/MyDrive/KIST/open/train/*/label.csv')
labels.sort()

In [29]:
label = pd.DataFrame()
for df in tqdm(labels):
  temp = pd.read_csv(df)
  label = pd.concat([label, temp], axis=0)

label = label.reset_index(drop=True)
label

100%|██████████| 75/75 [00:14<00:00,  5.10it/s]


In [23]:
train = pd.DataFrame()
for df in tqdm(metas):
  temp = pd.DataFrame(pd.read_csv(df).mean().fillna(0)).T
  train = pd.concat([train, temp], axis=0)

train = train.reset_index(drop=True)
train

  0%|          | 0/1592 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 1592/1592 [00:24<00:00, 64.03it/s]


In [35]:
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    sensitivity = 35
    lower_hsv = np.array([60 - sensitivity, 50, 0])
    upper_hsv = np.array([60 + sensitivity, 255, 255])

    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

In [ ]:
area = []
for img in tqdm(images):
    image = cv2.imread(img)
    image_mask = create_mask_for_plant(image)
    per = len(np.where(image_mask>0)[0])/(len(image_mask)*len(image_mask[0]))
    area.append(per)

 43%|████▎     | 691/1592 [15:16<15:07,  1.01s/it]

In [ ]:
train['area'] = area

In [ ]:
train.to_csv('/content/drive/MyDrive/KIST/train.csv', index=False)